In [1]:
import pandas as pd
import numpy as np
import altair as alt
import plotly.express as px
import pickle
import math

In [2]:
folder = 'res_Assis2009' #'res_Neurips' 'res_AssisChallenge'

In [3]:
for i in range(5):
    tmp = pd.read_csv(f'./new/{folder}/results/irt_all_results_new_{i}.csv')
    try:
        df = pd.concat([df,tmp])
    except:
        df = tmp

In [4]:
nb_learners = df.nb_learners.unique()
nb_tests = df.nb_tests.unique()

k = df.K.unique()
m = df.M.unique()

variants = df.variant.unique()

In [5]:
setting = 'all'

df_ = df[df.apt != 'no'].drop(columns=['apt'])
df_ = df_[df_.variant != setting].drop(columns=['variant'])

df_ = df_[~df_.method.isin(['OursR_S','OursV_S', 'PerfSort', 'OursR'])]

df_ = df_.replace('single_DKT', 'SingleDKT')
df_ = df_.replace('single_IRT', 'SingleIRT')
df_ = df_.replace('Greedy', '  ALTER')
df_ = df_.replace('RandomGroup', 'RGroup')
df_ = df_.replace('RandomPerf', 'RPerf')
df_ = df_.replace('OursV', '  GLAC')
df_ = df_.replace('kmeans', 'K-means')
df_ = df_.replace('AntiVar', ' V-Clustering')
df_ = df_.replace('AntiDiv', ' D-Clustering')
df_ = df_.replace('Balanced', ' BALANCED')

Both Objectives: Group Learning and Expected Performance

Evolution of batch size and number of learners

In [11]:
m = 10

groups = []
exps = []

hahaha = []

df_2 = df_[(df_.method != 'SingleDKT') & (df_.method != 'SingleIRT')]

lol = True

for k in [10,50]:
    all_group = None
    all_exp = None
    
    first = True
    
    for nb_l in [100, 1000]:
        df_temp = df_2[(df_2.nb_learners == nb_l) & (df_2.K == k) & (df_2.M == m)]

        if len(df_temp) == 0:
            continue
        
        if lol:
            y = "Exp Perf"
        else:
            y = ""
        
        if folder == 'res_Assis2009':
            if k == 10 and nb_l == 100:
                sc = 0.8
            elif k == 10 and nb_l == 1000:
                sc = 2.4
            elif k == 50 and nb_l == 100:
                sc = 0.8
            else:
                sc = 2.8
        else:
            if k == 10 and nb_l == 100:
                sc = 0.6
            elif k == 10 and nb_l == 1000:
                sc = 4
            elif k == 50 and nb_l == 100:
                sc = 0.8
            else:
                sc = 4
        
        df_temp.loc[:, 'expectedPerf'] = df_temp['expectedPerf'].apply(lambda x: x / 1000.0)
        
        chart_exp = alt.Chart(df_temp).mark_bar().encode(
            alt.X('method:N',axis=alt.Axis(title="",labels=False, titleFontSize=26,
                                       titleFont = 'Time New Roman', titleFontStyle='italic')
             ),
        
            alt.Y('mean(expectedPerf)', scale=alt.Scale(domain=[0,sc]),
              axis=alt.Axis(title=y, values=[sc/2, sc],
                            labelFontSize=25,labelFont='Time New Roman', labelFontWeight='bold',
                            titleFontSize=26, titleFont = 'Time New Roman', titleFontStyle='italic', 
                            titleFontWeight='bold',
                            grid=False,labelAngle=0)
             ),
            
            color = alt.Color('method:N', title='', #legend = None
                        legend=alt.Legend(orient='bottom', titleFontSize=25, columns=4, labelFontSize=23, 
                        titleFontStyle='italic',labelFontStyle='bold',titleFont = 'Time New Roman',
                        labelFont='Time New Roman',symbolSize=500) 
                       )
            
        ).properties(
            title = [f'Learners : {nb_l}', f'K : {k}'],
            width = 80,
            height = 160,
        )
        
        error_bars_experf = alt.Chart(df_temp).mark_errorbar(ticks=True).encode(
            x='method:N',
            y=alt.Y('expectedPerf',title=y),
        )
        
        chart_exp = chart_exp+error_bars_experf
        
        hahaha.append(chart_exp)
        
        if lol:
            y = "Group Learning"
        else:
            y = ""
        
        if folder == 'res_Assis2009':
            if k == 10 and nb_l == 100:
                sc = 40
            elif k == 10 and nb_l == 1000:
                sc = 60
            elif k == 50 and nb_l == 100:
                sc = 100
            else:
                sc = 280
        else:
            if k == 10 and nb_l == 100:
                sc = 400
            elif k == 10 and nb_l == 1000:
                sc = 240
            elif k == 50 and nb_l == 100:
                sc = 1000
            else:
                sc = 1200
        
        #df_temp.learnGroupAll = df_temp.learnGroupAll/100
        df_temp.loc[:, 'learnGroupAll'] = df_temp['learnGroupAll'].apply(lambda x: x / 100.0)
        
        sc = sc/100.0  
        
        chart_lg = alt.Chart(df_temp).mark_bar().encode(
            alt.X('method:N',axis=alt.Axis(title="",labels=False, titleFontSize=26,
                                       titleFont = 'Time New Roman', titleFontStyle='italic')
             ),
        
            alt.Y('mean(learnGroupAll)', scale=alt.Scale(domain=[0,sc]),
              axis=alt.Axis(title=y, values=[sc, sc/2.],
                            labelFontSize=25,labelFont='Time New Roman', labelFontWeight='bold',
                            titleFontSize=26, titleFont = 'Time New Roman', titleFontStyle='italic', 
                            titleFontWeight='bold',
                            grid=False,labelAngle=0)
             ),
            
            color = alt.Color('method:N', title='', #legend = None
                        legend=alt.Legend(orient='bottom', titleFontSize=25, columns=4, labelFontSize=23, 
                        titleFontStyle='italic',labelFontStyle='bold',titleFont = 'Time New Roman',
                        labelFont='Time New Roman',symbolSize=500) 
                       )
        ).properties(
            title = [f'Learners : {nb_l}', f'K : {k}'],
            width = 80,
            height = 160,
        )
        
        error_bars_gl = alt.Chart(df_temp).mark_errorbar(ticks=True).encode(
            x='method:N',
            y=alt.Y('learnGroupAll',title=y),
        )
        
        chart_lg = chart_lg+error_bars_gl
            
        if first:
            all_charts_exp = chart_exp
            all_charts_lg = chart_lg
            first = False
        else:
            all_charts_exp = alt.hconcat(all_charts_exp, chart_exp, spacing=5)
            all_charts_lg = alt.hconcat(all_charts_lg, chart_lg, spacing=5)
    
        if lol:
            lol = False
            
    exps.append(all_charts_exp)
    groups.append(all_charts_lg)

In [12]:
alt.hconcat(exps[0], exps[1], spacing=1).configure_title(fontSize=15, offset=-10)

alt.HConcatChart(...)

In [13]:
alt.hconcat(groups[0], groups[1], spacing=10).configure_title(fontSize=15, offset=-10)

alt.HConcatChart(...)

Evolution of group size

In [14]:
k = 50
nb_t = 50

groups = []
exps = []

df_2 = df_[(df_.method != 'SingleDKT') & (df_.method != 'SingleIRT')]

lol = True

for m in [5,10,20]:
    all_group = None
    all_exp = None
    
    first = True

    for nb_l in [1000]:#nb_learners:
        df_temp = df_2[(df_2.nb_learners == nb_l) & (df_2.nb_tests == nb_t) &
                     (df_2.K == k) & (df_2.M == m)]


        #df_gain = df_temp[df_temp.improve != 0]

        if len(df_temp) == 0:
            continue
        
        if lol:
            y = "Exp Perf (e3)"
        else:
            y = ""
        
        if folder == 'res_Assis2009':
            if m == 5:
                sc = 1.4
            elif m == 10:
                sc = 2.8
            else:
                sc = 6
        else:
            if m == 5:
                sc = 1.8
            elif m == 10:
                sc = 4
            else:
                sc = 8.
            
        df_temp.expectedPerf = df_temp.expectedPerf/1000
        
        chart_exp = alt.Chart(df_temp).mark_bar().encode(
            alt.X('method:N',axis=alt.Axis(title="",labels=False, titleFontSize=26,
                                       titleFont = 'Time New Roman', titleFontStyle='italic')
             ),
        
            alt.Y('mean(expectedPerf)', scale=alt.Scale(domain=(0,sc)),
              axis=alt.Axis(title=y, values=[sc/2,sc],
                            labelFontSize=25,labelFont='Time New Roman', labelFontWeight='bold',
                            titleFontSize=26, titleFont = 'Time New Roman', titleFontStyle='italic', 
                            titleFontWeight='bold', 
                            grid=False,labelAngle=0)
             ),
            color = alt.Color('method:N', title='', #legend = None
                        legend=alt.Legend(orient='bottom', titleFontSize=25, columns=4, labelFontSize=23, 
                        titleFontStyle='italic',labelFontStyle='bold',titleFont = 'Time New Roman',
                        labelFont='Time New Roman',symbolSize=500) 
                       )
        ).properties(
            title = f'm : {m}',
            width = 130,
            height = 220
        )
        
        error_bars_exp = alt.Chart(df_temp).mark_errorbar(ticks=True).encode(
            x='method:N',
            y=alt.Y('expectedPerf',title=y),
        )
        
        chart_exp = chart_exp+error_bars_exp
        
        
        chart_lg = alt.Chart(df_temp).mark_bar().encode(
            x = 'method',
            y = 'mean(learnGroup)',
            color = 'method'
        ).properties(
            title = f'm : {m}',
            width = 130,
            height = 220
        )
        
        error_bars_lg = alt.Chart(df_temp).mark_errorbar(ticks=True).encode(
            x='method:N',
            y=alt.Y('learnGroup',title=y),
        )
        
        chart_lg = chart_lg+error_bars_lg
        
        if lol:
            lol = False
            
        if first:
            all_charts_exp = chart_exp
            all_charts_lg = chart_lg
            first = False
        else:
            all_charts_exp = all_charts_exp | chart_exp
            all_charts_lg = all_charts_lg | chart_lg
    
    exps.append(all_charts_exp)
    groups.append(all_charts_lg)

/var/folders/pw/s1kjls0j5b50xd_265_cy2x80000gp/T/ipykernel_35707/855655411.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.expectedPerf = df_temp.expectedPerf/1000


In [15]:
(exps[0] | exps[1] | exps[2]).configure_title(fontSize=16, offset=-10)

alt.HConcatChart(...)

Skill Gain and Improved Skills + Time

Evolution of batch size and number of learners

In [16]:
k = 10

chrt = []

df_2 = df_.copy()#[(df_.method != 'SingleDKT') & (df_.method != 'SingleIRT')]


for nb_l in [100,1000]:
    all_charts = None
        
    for m in [5,20]:
        
        if m == 5:
            first = True
        else:
            first = False
        
        df_temp = df_2[(df_2.nb_learners == nb_l) & (df_2.K == k) & (df_2.M == m)]

        if len(df_temp) == 0:
            continue

        if first:
            y = "Skill Gain"
            z = ["Skills (%)"]
        else:
            y,z = "Skill Gain", ''
            
        
        if folder == 'res_Assis2009':
            if m == 5 and nb_l == 100:
                sc = 0.25
                sc_deb = 0.04

                sc2_deb = 0.005
                sc2_fin = 0.042
            elif m == 5 and nb_l == 1000:
                sc = 0.1
                sc_deb = 0.03

                sc2_deb = 0.007
                sc2_fin = 0.02
            elif m == 20 and nb_l == 100:
                sc = 0.16
                sc_deb = 0.04

                sc2_deb = 0.008
                sc2_fin = 0.02
            else:
                sc = 0.06
                sc_deb = 0.04

                sc2_deb = 0.0095
                sc2_fin = 0.013
        
        else:
            if m == 5 and nb_l == 100:
                sc = 0.6
                sc_deb = 0.3

                sc2_deb = 0.07
                sc2_fin = 0.105
            elif m == 5 and nb_l == 1000:
                sc = 0.2
                sc_deb = 0.05

                sc2_deb = 0.015
                sc2_fin = 0.045
            elif m == 20 and nb_l == 100:
                sc = 0.45
                sc_deb = 0.35

                sc2_deb = 0.075
                sc2_fin = 0.095
            else:
                sc = 0.14
                sc_deb = 0.09

                sc2_deb = 0.02
                sc2_fin = 0.03
        
        chart_len_kc = alt.Chart(df_temp).mark_point(size=150, opacity=1, filled=True, color= 'Black').encode(
            x = alt.X('mean(gain_p_skill_avg)', scale=alt.Scale(domain=[sc2_deb,sc2_fin]),
                      axis=alt.Axis(title=y, titleFontSize=26,labelFontSize=20,
                                    titleFont = 'Time New Roman',
                                    titleFontStyle='italic')
                     ),

            y = alt.Y('mean(improve_p_skill_avg)', scale=alt.Scale(domain=[sc_deb,sc]),
                      axis = alt.Axis(title=z, labelFontSize=25,labelFont='Time New Roman', 
                                      labelFontWeight='bold',titleFontSize=26, titleFont = 'Time New Roman', 
                                      titleFontStyle='italic', titleFontWeight='bold',
                                      labelAngle=0, tickCount=3
                                     )
                     ),

            shape = alt.Shape('method', legend=None),
            color = alt.Color('method:N', title='', #legend = None
                        #[legend=alt.Legend(orient='bottom', titleFontSize=25, columns=5, labelFontSize=23, 
                        #titleFontStyle='italic',labelFontStyle='bold',titleFont = 'Time New Roman',
                        #labelFont='Time New Roman',symbolSize=500) 
                       )
        ).properties(
            width=250,
            height=120,
            #title = [f'Learners : {nb_l}',f'm : {m}']
            title = [f'm : {m}']
        )

        chart_text = alt.Chart(df_temp).mark_text(size=14).encode(
            x = alt.X('mean(gain_p_skill_avg)', title='Skill Gain'),
            y = alt.Y('mean(improve_p_skill_avg)', title='% Learners'),
            text = alt.Text('method'),
        ).properties(
            width=400,
            height=200,
            title = f'Learners : {nb_l} - Size batch : {m}'
        )
        
        chart = chart_len_kc#+chart_text

        if all_charts is None:
            all_charts = chart
        else:
            all_charts = alt.hconcat(all_charts, chart, spacing=5)
    
    chrt.append(all_charts)

In [17]:
#Learners = 100
chrt[0].configure_title(fontSize=16, offset=-10)

alt.HConcatChart(...)

In [18]:
#Learners = 1000
chrt[1].configure_title(fontSize=16, offset=-10)

alt.HConcatChart(...)

Evolution of time as function of m

In [23]:
k = 10
times = []

df_2 = df_[(df_.method != 'SingleDKT') & (df_.method != 'SingleIRT')]

all_charts_time = None
first = True

for nb_l in nb_learners:
    df_temp = df_2[(df_2.nb_learners == nb_l) & (df_2.K == k)]
    
    if len(df_temp) == 0:
        continue

    chart_time = alt.Chart(df_temp).mark_line(interpolate='natural').encode(
         alt.X('M:N',axis=alt.Axis(title="m", labelFontSize=25,labelFont='Time New Roman', labelFontWeight='bold',
                        titleFontSize=26, titleFont = 'Time New Roman', titleFontStyle='italic', 
                        titleFontWeight='bold', 
                        tickCount= 5,grid=False,labelAngle=0)
         ),

        alt.Y('mean(time)',
          axis=alt.Axis(title="Time (sec)", #values=[10,7.5,5,2.5],
                        labelFontSize=25,labelFont='Time New Roman', labelFontWeight='bold',
                        titleFontSize=26, titleFont = 'Time New Roman', titleFontStyle='italic', 
                        titleFontWeight='bold',labelAngle=0)
         ),

        color = alt.Color('method:N', title='', #legend = None
                        legend=alt.Legend(orient='bottom', titleFontSize=25, columns=4, labelFontSize=23, 
                        titleFontStyle='italic',labelFontStyle='bold',titleFont = 'Time New Roman',
                        labelFont='Time New Roman',symbolSize=500) 
                       ),
        tooltip=['average(time)']
    ).properties(
        width=250,
        height=150,
    ).interactive()


    if first:
        all_charts_time = chart_time
        first = False
    else:
        all_charts_time = all_charts_time | chart_time

    times.append(chart_time)
    
t_1 = times[2]

Evolution of group size

In [26]:
m = 5

chrt = []

df_2 = df_.copy()

for k in [10,50,100]:
    all_charts = None
    
    if k == 10:
        first = True
    else:
        first = False

    for nb_l in [1000]:
        df_temp = df_2[(df_2.nb_learners == nb_l) & (df_2.K == k) & (df_2.M == m)]

        if len(df_temp) == 0:
            continue
        
        if first:
            y = "Skill Gain"
            z = ["Skills (%)"]
        else:
            y,z = "Skill Gain", ''
            
        
        if folder == 'res_Assis2009':
            if k == 10:
                sc = 0.1
                sc_deb = 0.03
                
                sc2 = 0.02
                sc2_deb = 0.006
                
            elif k == 50:
                sc = 0.1
                sec_deb = 0.06
            else:
                sc = 0.1
                sc_deb = 0.02
                
                sc2 = 0.018
                sc2_deb = 0.006
        else:
            if k == 10:
                sc = 0.2
                sc_deb = 0.08
                
                sc2 = 0.045
                sc2_deb = 0.016
                
            else:
                sc = 0.2
                sc_deb = 0.08
                
                sc2 = 0.042
                sc2_deb = 0.016

        
        chart_len_kc = alt.Chart(df_temp).mark_point(size=150, opacity=1, filled=True, color= 'Black').encode(
            x = alt.X('mean(gain_p_skill_avg)', scale=alt.Scale(domain=[sc2_deb, sc2]),
                      axis=alt.Axis(title=y, titleFontSize=26, 
                                                              labelFontSize=20,titleFont = 'Time New Roman', 
                                                              titleFontStyle='italic')
                     ),

            y = alt.Y('mean(improve_p_skill_avg)', scale=alt.Scale(domain=[sc_deb,sc]),
                      axis = alt.Axis(title=z, labelFontSize=25,labelFont='Time New Roman', 
                                      labelFontWeight='bold',titleFontSize=26, titleFont = 'Time New Roman', 
                                      titleFontStyle='italic', titleFontWeight='bold', labelAngle=0
                                     )
                     ),

            shape = alt.Shape('method', legend=None),
            color = alt.Color('method:N', title='', #legend = None
                              legend=alt.Legend(orient='bottom', titleFontSize=25, columns=5, labelFontSize=23, 
                                                titleFontStyle='italic',labelFontStyle='bold',
                                                titleFont = 'Time New Roman', labelFont='Time New Roman',
                                                symbolSize=500) 
                       )
        ).properties(
            width=250,
            height=120,
            title = f'K : {k}'
        )

        chart_text = alt.Chart(df_temp).mark_text(size=14).encode(
            x = alt.X('mean(gain_p_skill_avg)', title='Skill Gain'),
            y = alt.Y('mean(improve_p_skill_avg)', title='% Learners'),
            text = alt.Text('method'),
        ).properties(
            width=400,
            height=200,
            title = f'Learners : {nb_l} - Size batch : {m}'
        )
        
        chart = chart_len_kc#+chart_text

        if all_charts is None:
            all_charts = chart
        else:
            all_charts = alt.vconcat(all_charts, chart, spacing=15)
    
    chrt.append(all_charts)

In [27]:
alt.hconcat(chrt[0], chrt[2], spacing=5).configure_title(fontSize=16, offset=-10)

alt.HConcatChart(...)

Evolution of time as a function of K

In [28]:
m = 10
times = []

df_2 = df_[(df_.method != 'SingleDKT') & (df_.method != 'SingleIRT')]

all_charts_time = None

first = True

for nb_l in nb_learners:
    df_temp = df_2[(df_2.nb_learners == nb_l) & (df_2.M == m)]

    if len(df_temp) == 0:
        continue

    chart_time = alt.Chart(df_temp).mark_line(interpolate='natural').encode(
         alt.X('K:N',axis=alt.Axis(title="k", labelFontSize=25,labelFont='Time New Roman', labelFontWeight='bold',
                        titleFontSize=26, titleFont = 'Time New Roman', titleFontStyle='italic', 
                        titleFontWeight='bold', 
                        tickCount= 5,grid=False,labelAngle=0)
         ),

        alt.Y('mean(time)',
          axis=alt.Axis(title="",
                        labelFontSize=25,labelFont='Time New Roman', labelFontWeight='bold',
                        titleFontSize=26, titleFont = 'Time New Roman', titleFontStyle='italic', 
                        titleFontWeight='bold', 
                        labelAngle=0)
         ),

        color = alt.Color('method:N', title='', #legend = None
                        legend=alt.Legend(orient='bottom', titleFontSize=25, columns=4, labelFontSize=23, 
                        titleFontStyle='italic',labelFontStyle='bold',titleFont = 'Time New Roman',
                        labelFont='Time New Roman',symbolSize=700, symbolStrokeWidth=5) 
                       )
    ).properties(
        width=250,
        height=150,
    )


    if first:
        all_charts_time = chart_time
        first = False
    else:
        all_charts_time = all_charts_time | chart_time

    times.append(chart_time)

In [29]:
t_1 | times[2]

alt.HConcatChart(...)

Evolution of time as a function of number of learners

In [30]:
m = 10
k = 10

times = []

df_2 = df_[(df_.method != 'SingleDKT') & (df_.method != 'SingleIRT')]

all_charts_time = None

first = True

df_temp = df_2[(df_2.K == k) & (df_2.M == m)]

chart_time = alt.Chart(df_temp).mark_line().encode(
     alt.X('nb_learners:N',axis=alt.Axis(title="#Learners", labelFontSize=25,labelFont='Time New Roman', labelFontWeight='bold',
                    titleFontSize=26, titleFont = 'Time New Roman', titleFontStyle='italic', 
                    titleFontWeight='bold', 
                    tickCount= 5,grid=False,labelAngle=0)
     ),

    alt.Y('mean(time)',
      axis=alt.Axis(title="Time",
                    labelFontSize=25,labelFont='Time New Roman', labelFontWeight='bold',
                    titleFontSize=26, titleFont = 'Time New Roman', titleFontStyle='italic', 
                    titleFontWeight='bold', 
                    tickCount= 5,grid=False,labelAngle=0)
     ),

    color = alt.Color('method:N', title='', #legend = None
                    legend=alt.Legend(orient='bottom', titleFontSize=25, columns=4, labelFontSize=23, 
                    titleFontStyle='italic',labelFontStyle='bold',titleFont = 'Time New Roman',
                    labelFont='Time New Roman',symbolSize=700, symbolStrokeWidth=5) 
                   )
).properties(
    title = f'Learners = {nb_l}',
    width=300,
    height=180,
)


if first:
    all_charts_time = chart_time
    first = False
else:
    all_charts_time = all_charts_time | chart_time

times.append(chart_time)

In [31]:
times[0]

alt.Chart(...)

Potential Learning

In [33]:
df_ = df[df.variant != setting]
df_ = df_[~df_.method.isin(['OursR_S','OursV_S', 'PerfSort', 'OursR'])]

df_ = df_.replace('single_DKT', 'SingleDKT')
df_ = df_.replace('single_IRT', 'SingleIRT')

df_ = df_.replace('RandomGroup', 'RGroup')
df_ = df_.replace('RandomPerf', ' RPerf')

df_ = df_.replace('Greedy', ' Alternate')
df_ = df_.replace('OursV', ' AntiClustering')
df_ = df_.replace('kmeans', 'Kmeans')
df_ = df_.replace('AntiVar', ' V-Clustering')
df_ = df_.replace('AntiDiv', ' D-Clustering')
df_ = df_.replace('Balanced', ' BALANCED')

df_with = df_[df_.apt == 'yes'][['nb_learners','nb_tests','K','M','method','gain_p_skill_avg',\
                                 'improve_p_skill_avg','time','at_least','coverage_skills']]
df_without = df_[df_.apt == 'no'][['nb_learners','nb_tests','K','M','method','gain_p_skill_avg',\
                                   'improve_p_skill_avg','time','at_least','coverage_skills']]

df_ = pd.merge(df_with,df_without, on=['nb_learners','nb_tests','K','M','method'])

In [34]:
df_['gain_apt'] = df_.gain_p_skill_avg_x - df_.gain_p_skill_avg_y
df_['improve_apt'] = df_.improve_p_skill_avg_x - df_.improve_p_skill_avg_y
df_['cov_apt'] = df_.coverage_skills_x - df_.coverage_skills_y
df_['at_apt'] = df_.at_least_x - df_.at_least_y
df_['time_apt'] = df_.time_x - df_.time_y

In [35]:
k = 10

improv = []
gains = []
times = []
covs = []
ats = []

for m in [5]:    
    first = True

    for nb_l in [1000]:
        df_temp = df_[(df_.nb_learners == nb_l) & (df_.nb_tests == nb_t) &
                     (df_.K == k) & (df_.M == m)]

        if len(df_temp) == 0:
            continue

        
        chart_impr = alt.Chart(df_temp).mark_bar().encode(
            alt.X('method:N',axis=alt.Axis(title="",labels=False, titleFontSize=26,
                                       titleFont = 'Time New Roman', titleFontStyle='italic')
             ),
        
            alt.Y('mean(improve_apt)', scale=alt.Scale(domain=[0,0.16]),
              axis=alt.Axis(title="Improved Skills", values=[0.08, 0.16],
                            labelFontSize=25,labelFont='Time New Roman', labelFontWeight='bold',
                            titleFontSize=26, titleFont = 'Time New Roman', titleFontStyle='italic', 
                            titleFontWeight='bold', 
                            grid=False,labelAngle=0)
             ),
            color = alt.Color('method:N', title='', #legend = None
                       )
            
        ).properties(
            width = 130,
            height = 220,
        )
        
        error_bars_impr = alt.Chart(df_temp).mark_errorbar(ticks=True).encode(
            x='method:N',
            y=alt.Y('improve_apt',title="Improved Skills"),
        )
        
        chart_impr = chart_impr+error_bars_impr
        
        

        chart_gain = alt.Chart(df_temp).mark_bar().encode(
             alt.X('method:N',axis=alt.Axis(title="",labels=False, titleFontSize=26,
                                       titleFont = 'Time New Roman', titleFontStyle='italic')
             ),
        
            alt.Y('mean(gain_apt)', scale=alt.Scale(domain=[0,0.04]),
              axis=alt.Axis(title="Skill Gain", values=[0.02, 0.04],
                            labelFontSize=25,labelFont='Time New Roman', labelFontWeight='bold',
                            titleFontSize=26, titleFont = 'Time New Roman', titleFontStyle='italic', 
                            titleFontWeight='bold', 
                            grid=False,labelAngle=0)
             ),
            
            color = alt.Color('method:N', title='', legend = None
                       )
        ).properties(
            width = 130,
            height = 220,
        )
        
        
        error_bars_gain = alt.Chart(df_temp).mark_errorbar(ticks=True).encode(
            x='method:N',
            y=alt.Y('gain_apt',title="Skill Gain"),
        )
        
        chart_gain = chart_gain+error_bars_gain
        
        
        chart_at = alt.Chart(df_temp).mark_bar().encode(
            alt.X('method:N',axis=alt.Axis(title="",labels=False, titleFontSize=26,
                                       titleFont = 'Time New Roman', titleFontStyle='italic')
             ),
        
            alt.Y('mean(at_apt)', scale=alt.Scale(domain=[0,0.22]),
              axis=alt.Axis(title="Improved Learners", values=[0.22, 0.11],
                            labelFontSize=25,labelFont='Time New Roman', labelFontWeight='bold',
                            titleFontSize=26, titleFont = 'Time New Roman', titleFontStyle='italic', 
                            titleFontWeight='bold', 
                            tickCount= 5,grid=False,labelAngle=0)
             ),
            
            color = alt.Color('method:N', title=''
                       )
        ).properties(
            width = 130,
            height = 220,
        )
        
        error_bars_at = alt.Chart(df_temp).mark_errorbar(ticks=True).encode(
            x='method:N',
            y=alt.Y('at_apt',title="Improved Learners"),
        )
        
        chart_at = chart_at+error_bars_at

        if first:
            all_charts_impr = chart_impr
            all_charts_gain = chart_gain
            all_charts_at = chart_at
            first = False
        else:
            all_charts_impr = all_charts_impr & chart_impr
            all_charts_gain = all_charts_gain & chart_gain
            all_charts_at = all_charts_at & chart_at
    
    all_charts_impr = all_charts_impr
    all_charts_gain = all_charts_gain
    all_charts_at = all_charts_at
    
    improv.append(all_charts_impr)
    gains.append(all_charts_gain)
    ats.append(all_charts_at)

In [36]:
gains[0] | improv[0]

alt.HConcatChart(...)

Improvement per Skills

In [37]:
df_3 = df.copy()
nb_tests = nb_tests[0]

In [38]:
k = 10
m = 5

for nb_learners in [1000]:
    for i, k in enumerate([10, 100]):
        df = df_3.copy()
        df = df[(df.K == k) & (df.M == m) & (df.nb_learners == nb_learners) & (df.nb_tests == nb_tests)].head(40)
        
        if k == 10:
            y = 'Skills'
        else:
            y = ''
            
        for simul in range(5):
            df_loc = df[['method','apt','variant']]

            simulation = f'__{simul}'

            with open(f'./new/{folder}/detailed/irt_res_{nb_learners}_{nb_tests}_{k}_{m}{simulation}', 'rb') as fp:
                d = pickle.load(fp)

            df_gains = []
            df_improves = []

            for key, value in d.items():
                for idx, (us, val) in enumerate(value.items()):
                    
                    skills = val[0]
                    improves = val[1]
                    gains = val[2]

                    if idx == 0:
                        all_gains = [g for g in gains]
                        all_improves = [g for g in improves]
                    else:
                        all_gains = [g+g2 for g, g2 in zip(all_gains,gains)]
                        all_improves = [g+g2 for g, g2 in zip(all_improves,improves)]

                all_gains = [g/len(value) for g in all_gains]
                all_improves = [g/len(value) for g in all_improves]

                df_gains.append(all_gains)
                df_improves.append(all_improves)

            nb_skills = len(df_gains[0])

            columns=[f'gain_skill_{i+1}' for i in range(nb_skills)]
            df_gains = pd.DataFrame(df_gains, columns=columns)

            df_loc.index = range(len(df_loc))
            df_gains.index = range(len(df_gains))

            df_loc = pd.concat([df_loc,df_gains],axis=1)

            columns=[f'improve_skill_{i+1}' for i in range(nb_skills)]
            df_improves = pd.DataFrame(df_improves, columns=columns)

            df_improves.index = range(len(df_improves))

            df_loc = pd.concat([df_loc,df_improves],axis=1)

            if simul == 0:
                df_all = df_loc.copy()
            else:
                df_all = pd.concat([df_all,df_loc],axis=0)
        
        for skill in range(nb_skills): 
            column = f'improve_skill_{skill+1}'
            column2 = f'gain_skill_{skill+1}'

            df_loc = df_all[['method','apt','variant',column,column2]]

            df_loc = df_loc.rename(columns={column:'improve_skill', column2:'gain_skill'})
            df_loc['skill'] = skill+1#f's{skill+1}'

            if skill == 0:
                df_skill = df_loc
            else:
                df_skill = pd.concat([df_skill,df_loc],axis=0)

        df_loc = df_skill.copy()

        df_loc = df_loc[df_loc.apt != 'no'].drop(columns=['apt'])
        df_loc = df_loc[df_loc.variant != setting].drop(columns=['variant'])
        df_loc = df_loc[~df_loc.method.isin(['OursR_S','OursV_S', 'PerfSort', 'OursR'])]

        df_loc = df_loc.replace('single_DKT', 'SingleDKT')
        df_loc = df_loc.replace('single_IRT', 'SingleIRT')
        df_loc = df_loc.replace('Greedy', '  ALTER')
        df_loc = df_loc.replace('OursV', '  GLAC')
        df_loc = df_loc.replace('kmeans', 'K-means')
        df_loc = df_loc.replace('AntiVar', ' V-Clustering')
        df_loc = df_loc.replace('AntiDiv', ' D-Clustering')
        df_loc = df_loc.replace('RandomGroup', 'RGroup')
        df_loc = df_loc.replace('RandomPerf', 'RPerf')
        df_loc = df_loc.replace('Balanced', ' BALANCED')
        
        df_loc = df_loc[~df_loc.method.isin(['SingleIRT', 'RGroup', 'RPerf', ' BALANCED'])]
        
        range_ = ['#4c78a8', '#f58518', '#72b7b2', '#54a24b', '#eeca3b', '#9d755d']
        
        #stroke='Black', strokeWidth=1
        
        chart = alt.Chart(df_loc).mark_circle(stroke='Black', strokeWidth=1).encode(
            alt.X('method:N', axis=alt.Axis(title="", labels=False)
                             ),

            alt.Y('skill',axis=alt.Axis(title=y, values=list(df_loc.skill.unique()),
                                        titleFontSize=26, labelFontSize=20,
                                        titleFont = 'Time New Roman', titleFontStyle='italic')
                 ),
            
            size = alt.Size("mean(improve_skill):Q", title='Rates (%):', #legend = None
                            legend=alt.Legend(orient='bottom', titleFontSize=25, columns=6, labelFontSize=23, 
                                              titleFontStyle='italic',labelFontStyle='bold',
                                              titleFont = 'Time New Roman',labelFont='Time New Roman',
                                              symbolSize=700, symbolStrokeWidth=5)
                           ),
            
            color = alt.Color('method', title='', scale=alt.Scale(range=range_), #legend = None,
                              legend=alt.Legend(orient='bottom', titleFontSize=25, columns=3, 
                                                          labelFontSize=23, titleFontStyle='italic',
                                                          labelFontStyle='bold',titleFont = 'Time New Roman',
                                                          labelFont='Time New Roman',symbolSize=700, 
                                                          symbolStrokeWidth=5)
                             ),

        ).properties(
            #title = [f'Learners : {nb_learners}'],
            title = [f'K : {k}'],
            
            width=280,
            height=170,
        )
            
        
        if i == 0:
            row_chart = chart
        else:
            row_chart = row_chart | chart
    
    if nb_learners == 1000:
        all_charts = row_chart
    else:
        all_charts = all_charts | row_chart

In [39]:
#m = 5
all_charts.configure_title(fontSize=16, offset=0)

alt.HConcatChart(...)

At least n skills

In [40]:
k = 10
m = 5

for nb_learners in [100,1000]:

    all_df = []

    for simul in range(5):
        simulation = f'__{simul}'

        with open(f'./new/{folder}/detailed/irt_res_{nb_learners}_{nb_tests}_{k}_{m}{simulation}', 'rb') as fp:
            d = pickle.load(fp)

        df = df_3.copy()
        df_loc = df[(df.K == k) & (df.M == m) & (df.nb_learners == nb_learners) & (df.nb_tests == nb_tests)].head(40)
        df_loc = df_loc[['method','apt','variant']].values

        df_at_least = []

        for iidx, (key, value) in enumerate(d.items()):
            at_least_all = list()

            other = df_loc[iidx]

            for idx, (us, val) in enumerate(value.items()):
                improves = sum(val[1])
                len_ = len(val[1])

                at_least = [improves > i for i in range(len_)]

                if idx == 0:
                    at_least_all = list(at_least)
                else:
                    at_least_all = [at_least_all[i]+at_least[i] for i in range(len_)]

            at_least_all = [list(other)+list([i+1, (nb/idx)*100]) + list([simul]) for i,nb in enumerate(at_least_all)]
            df_at_least.extend(at_least_all)

        all_df.extend(df_at_least)

    all_df = pd.DataFrame(all_df, columns=['method','apt','variant','at_least','nb','simul'])
    
    #Process
    df_loc = all_df[all_df.apt != 'no'].drop(columns=['apt'])
    df_loc = df_loc[df_loc.variant != setting].drop(columns=['variant'])

    df_loc = df_loc[~df_loc.method.isin(['OursR_S','OursV_S', 'PerfSort', 'OursR'])]

    df_loc = df_loc.replace('single_DKT', 'SingleDKT')
    df_loc = df_loc.replace('single_IRT', 'SingleIRT')
    df_loc = df_loc.replace('Greedy', '  ALTER')
    df_loc = df_loc.replace('OursV', '  GLAC')
    df_loc = df_loc.replace('kmeans', 'K-means')
    df_loc = df_loc.replace('AntiVar', ' V-Clustering')
    df_loc = df_loc.replace('AntiDiv', ' D-Clustering')

    df_loc = df_loc[~df_loc.method.isin(['SingleIRT', 'Balanced', 'RandomGroup', 'RandomPerf'])]
    
    range_ = ['#4c78a8', '#f58518', '#72b7b2', '#54a24b', '#eeca3b', '#9d755d']
    
    df_loc = df_loc.groupby(['at_least','method']).nb.mean().reset_index()
    
    if nb_learners == 100:
        y = ["Learners (%)"]
    else:
        y = ''
        
    chart_at_least = alt.Chart(df_loc).mark_line(interpolate='natural').encode(
        alt.X('at_least:Q',axis=alt.Axis(title=["n"], 
                                       labelFontSize=25,labelFont='Time New Roman', labelFontWeight='bold',
                                       titleFontSize=26, titleFont = 'Time New Roman', titleFontStyle='italic', 
                                       titleFontWeight='bold', grid=False, labelAngle=0)
             ),

        alt.Y('mean(nb)', 
              axis=alt.Axis(title=y,
                            labelFontSize=25,labelFont='Time New Roman', 
                            labelFontWeight='bold',titleFontSize=26, titleFont = 'Time New Roman', 
                            titleFontStyle='italic', titleFontWeight='bold',tickCount=5, labelAngle=0)
             ),

        color = alt.Color('method:N', title='', scale=alt.Scale(range=range_), legend=None,
                          #legend=alt.Legend(orient='bottom', titleFontSize=25, columns=3, labelFontSize=23, 
                          #                  titleFontStyle='italic',labelFontStyle='bold',
                          #                  titleFont = 'Time New Roman',labelFont='Time New Roman',
                          #                  symbolSize=700, symbolStrokeWidth=5) 
                         ),
    ).properties(
        title=f"Learners : {nb_learners}",
        width=240,
        height=180,
    )
    
    if nb_learners==100:
        all_chart = chart_at_least
    else:
        all_chart = all_chart | chart_at_least
        

In [41]:
all_chart.configure_title(fontSize=16, offset=-1)

alt.HConcatChart(...)

Diameter vs Pairwise

In [42]:
nb_learners = 1000
nb_tests = 50

k = 50
m = 5

In [47]:
for sim in range(5):
    
    simulation = f'_{sim}'
        
    with open(f'./new/{folder}/learners/irt_{nb_learners}_{nb_tests}_new{simulation}', 'rb') as fp:
        learners = pickle.load(fp)
        
    with open(f'./new/{folder}/groupings/irt_groupings_{nb_learners}_{nb_tests}_{k}_{m}_new{simulation}', 'rb') as fp:
        grouping_list = pickle.load(fp)

    with open(f'./new/{folder}/detailed/irt_res_{nb_learners}_{nb_tests}_{k}_{m}_{simulation}', 'rb') as fp:
        res = pickle.load(fp)    
    
    
    stds_diameter = []
    stds_pariwise = []

    column_skills = []
    variant = []
        
    for mth in ['GLAC']:
        
        if mth == 'GLAC':
            idx_d = 30
            idx_a = 32
        elif mth == 'ALTER':
            idx_d = 2
            idx_a = 6
        elif mth == 'Balanced':
            idx_d = 12
            idx_a = 14

        for grp_i in range(k):
            grp_d, _ = grouping_list[idx_d][grp_i]
            skills = learners[grp_d,:]
            std_grp_d = skills.std(0)
            stds_diameter.extend(std_grp_d)
            
            grp_a, _ = grouping_list[idx_a][grp_i]
            skills = learners[grp_a, :]
            std_grp_a = skills.std(0)
            stds_pariwise.extend(std_grp_a)

            column_skills.extend([i+1 for i in range(len(std_grp_a))])
            variant.extend([mth+'_' for i in range(len(std_grp_a))])
    
    df_d = pd.DataFrame(stds_diameter, columns=['S'])
    df_d['NS'] = column_skills
    df_d['Variant'] = 'Diameter'
    df_d['Variant'] = variant + df_d['Variant']

    df_p = pd.DataFrame(stds_pariwise, columns=['S'])
    df_p['NS'] = column_skills
    df_p['Variant'] = 'Pairwise'
    df_p['Variant'] = variant + df_p['Variant']
    
    if sim == 0:
        df_ = pd.concat([df_d,df_p])
    else:
        df_ = pd.concat([df_,df_d])
        df_ = pd.concat([df_,df_p])

In [48]:
df = df_.groupby(['Variant','NS']).S.mean().reset_index()
df.NS = df.NS.apply(lambda x: f's{x}')

In [50]:
fig = px.line_polar(df, r='S', theta='NS', color='Variant', line_dash='Variant')

fig.update_layout(
    title = "",
    font_size = 20,
    showlegend = True,
    font_color ='black',
    
    polar = dict(
        bgcolor = "white",
        angularaxis = dict(
            linewidth = 0,
            showline = True,
            gridcolor = "black",
        ),
        radialaxis = dict(
            showline = False,
            linewidth = 1,
            gridcolor = "black",
            ticklabelstep= 4,
        )
    ),
    
    paper_bgcolor = "#ffffff"
)

fig.update_traces(line=dict(width=2.6))

fig#.show()